In [2]:
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 1.1 ms (started: 2021-11-23 12:30:52 -08:00)


# Options Apps Project

## Goal
Creating a suite of equity option web apps which expands on, and was inspired by, [Harshit Tyagi’s](https://www.linkedin.com/in/tyagiharshit/) [article](https://www.kdnuggets.com/2021/09/-structured-financial-newsfeed-using-python-spacy-and-streamlit.html) on building a financial newsfeed app.

## Skills Involved
Web scraping, NLP, options price modeling, predictive modeling of volitility, and building apps with Streamlit.

## Imports

In [3]:
import sys, os, math
import requests
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import time

#  setting path
gparent = os.path.join(os.pardir)
sys.path.append(gparent)

from src import helper_functions as h

time: 15.9 s (started: 2021-11-23 12:30:52 -08:00)


## requests: Grabbing Headlines

In [4]:
# grabbing headlines
r = requests.get("http://feeds.marketwatch.com/marketwatch/topstories/")
r

<Response [200]>

time: 176 ms (started: 2021-11-23 12:31:08 -08:00)


## bs4: Saving Headlines

In [5]:
# saving headlines to a list
soup = BeautifulSoup(r.content, features='lxml')
headlines = soup.findAll('title')
print(f'{type(headlines)}\n')
print(headlines)

<class 'bs4.element.ResultSet'>

[<title>MarketWatch.com - Top Stories</title>, <title>MarketWatch.com - Top Stories</title>, <title>: Biden vows gasoline prices will drop ‘before long’ but not ‘overnight’ as he orders oil-reserve release</title>, <title>Futures Movers: Oil prices end higher after U.S. Strategic Petroleum Reserve release</title>, <title>Brett Arends's ROI: Congress loves rich dead people</title>, <title>The Margin: J.R.R. Tolkien estate to ‘JRR Token’ cryptocurrency: You shall not pass</title>, <title>The Moneyist: My father is a cantankerous millionaire and hoarder. He cut my sister out of his will after a blazing row. How can I help her — without incurring a huge tax bill?</title>, <title>Commodities Corner: Why oil prices jumped despite the U.S. tapping the Strategic Petroleum Reserve</title>, <title>Coronavirus Update: WHO warns Europe COVID death toll could exceed 2.2 million by March, and U.S. warns against travel to Germany and Denmark</title>, <title>: Build Ba

### Grabbing Headline String
Grabbing headline and printing headline and isolated text.

In [6]:
# grabbing test headline
test_headline = headlines[2]
print(f'{test_headline}\n')
print(test_headline.text)

<title>: Biden vows gasoline prices will drop ‘before long’ but not ‘overnight’ as he orders oil-reserve release</title>

: Biden vows gasoline prices will drop ‘before long’ but not ‘overnight’ as he orders oil-reserve release
time: 998 µs (started: 2021-11-23 12:31:08 -08:00)


## spaCy: Tests
Testing tokenization and name extraction on a single string.

### Loading Model

In [7]:
# loading spacy model
nlp = spacy.load('en_core_web_sm')

time: 685 ms (started: 2021-11-23 12:31:08 -08:00)


### Tokenizing String
Tokenizing the test headline.

In [8]:
# checking the test case
processed_hline = nlp(test_headline.text)
print(f'{test_headline}\n')
for token in processed_hline:
  print(token)

<title>: Biden vows gasoline prices will drop ‘before long’ but not ‘overnight’ as he orders oil-reserve release</title>

:
Biden
vows
gasoline
prices
will
drop
‘
before
long
’
but
not
‘
overnight
’
as
he
orders
oil
-
reserve
release
time: 12.8 ms (started: 2021-11-23 12:31:09 -08:00)


## Saving List of Tokenized Headlines

In [9]:
processed_hlines = [nlp(headlines[i].text) for i in range(len(headlines))]
for line in processed_hlines:
    print(line)

MarketWatch.com - Top Stories
MarketWatch.com - Top Stories
: Biden vows gasoline prices will drop ‘before long’ but not ‘overnight’ as he orders oil-reserve release
Futures Movers: Oil prices end higher after U.S. Strategic Petroleum Reserve release
Brett Arends's ROI: Congress loves rich dead people
The Margin: J.R.R. Tolkien estate to ‘JRR Token’ cryptocurrency: You shall not pass
The Moneyist: My father is a cantankerous millionaire and hoarder. He cut my sister out of his will after a blazing row. How can I help her — without incurring a huge tax bill?
Commodities Corner: Why oil prices jumped despite the U.S. tapping the Strategic Petroleum Reserve
Coronavirus Update: WHO warns Europe COVID death toll could exceed 2.2 million by March, and U.S. warns against travel to Germany and Denmark
: Build Back Better — what’s in it for seniors?
Next Avenue: How will we make the most of an extra 30 years of life?
The Margin: Bezos’s Blue Origin sending Michael Strahan to space
time: 80.4 ms

## Getting Org Names From Headlines
Visualizing named entities (real world objects) in the headlines and creating a set of organiziations.

In [10]:
# pulling company name tokens from headlines
companies = []
for title in processed_hlines:
    doc = nlp(title.text)
    if len(doc.ents) != 0:
        displacy.render(doc, style='ent')
    else:
        pass
    for token in doc.ents:
        if token.label_ == 'ORG':
            companies.append(token.text)
        else:
            pass
companies = set(companies)
print(companies)

{'Petroleum Reserve', 'the Strategic Petroleum Reserve', 'Congress'}
time: 117 ms (started: 2021-11-23 12:31:09 -08:00)


## Scraping S&P 500 Stock Table w/Requests & BeautifulSoup

In [11]:
# scraping S&P wikipedia page
r = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# parsing the html
soup = BeautifulSoup(r.text, 'lxml')

# extracting the table
table = soup.find('table', {'class': 'wikitable sortable'})

# printing row with first stock
print(table.findAll('tr')[1:2])

[<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M</a></td>
<td><a class="external text" href="https://www.sec.gov/edgar/browse/?CIK=66740" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td>
<td>1976-08-09</td>
<td>0000066740</td>
<td>1902
</td></tr>]
time: 581 ms (started: 2021-11-23 12:31:09 -08:00)


### Ticker Symbols
Grabbing the ticker symbol from the first cell of each row.

In [12]:
# making list of symbols
symbols = [row.findAll('td')[0].text for row in table.findAll('tr')[1:]]

# checking length
print(f'List Length: {len(symbols)} \n')

# checking first 5 symbols
print(f'First five symbols: {symbols[:5]}')

List Length: 505 

First five symbols: ['MMM\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n']
time: 15.9 ms (started: 2021-11-23 12:31:10 -08:00)


The length of the list looks good, but we have extra characters we need to strip from the symbols.

In [13]:
# stripping new line character from the strings 
symbols = list(map(lambda s: s.strip(), symbols))

# checking first 5 symbols
print(symbols[:5])

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']
time: 1.22 ms (started: 2021-11-23 12:31:10 -08:00)


Looks good. We can apply the same technique to the second cell of the row to scrape and save the company name, and to the fifth cell to grab the company industry. 

### Names
Grabbing the company name from the second cell of each row.

In [14]:
# making list of names
names = [row.findAll('td')[1].text for row in table.findAll('tr')[1:]]

# checking first five names
print(names[:5])

['3M', 'Abbott Laboratories', 'AbbVie', 'Abiomed', 'Accenture']
time: 20.9 ms (started: 2021-11-23 12:31:10 -08:00)


## S&P 500 Dataframe
Creating a data frame of stocks in the S&P 500 index.

In [15]:
# making a data dictionary
data = {'Company Name': names, 'Symbol': symbols}

# creating data frame from the data
stocks_df = pd.DataFrame.from_dict(data)

# checking shape and first five rows
print(stocks_df.shape)
stocks_df.head()

(505, 2)


,Company Name,Symbol
0,3M,MMM
1,Abbott Laboratories,ABT
2,AbbVie,ABBV
3,Abiomed,ABMD
4,Accenture,ACN


time: 18.1 ms (started: 2021-11-23 12:31:10 -08:00)


### Checking yf Stock Info Dictionary Keys
Checking the dictionary keys available for pulling stock information.

In [16]:
# instantiating a ticker object
ACN = yf.Ticker('ACN')

time: 3.63 ms (started: 2021-11-23 12:31:10 -08:00)


In [17]:
# checking keys
ACN.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskR

time: 3.04 s (started: 2021-11-23 12:31:10 -08:00)


### Data for Stocks in the News
Creating a data frame of price and dividend information for S&P 500 stocks in the news in the following steps:

- Create a data dictionary from the list of companies in the headlines.

- Create a data frame from data dictionary.

In [18]:
# creating empty stock info dictionary
stock_data = {
    'Company': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    '52wkHigh': [],
    '52wkLow': [],
    'dividendRate': []
    
}

# loading stocks from s&p dataframe and appending data from yf
for company in companies:

    try:
        if stocks_df['Company Name'].str.contains(company).sum():
            symbol = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Symbol'].values[0]
            org_name = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Company Name'].values[0]
            stock_data['Company'].append(org_name)
            stock_data['Symbol'].append(symbol)
            stock_info = yf.Ticker(symbol).info
            stock_data['currentPrice'].append(stock_info['currentPrice'])
            stock_data['dayHigh'].append(stock_info['dayHigh'])
            stock_data['dayLow'].append(stock_info['dayLow'])
            stock_data['52wkHigh'].append(stock_info['fiftyTwoWeekHigh'])
            stock_data['52wkLow'].append(stock_info['fiftyTwoWeekLow'])            
            
            # converting dividend None types to floats
            dividend = stock_info['dividendRate']
            if dividend != None:
                dividend = dividend
            else:
                dividend = 0
            stock_data['dividendRate'].append(dividend)
        else:
            pass
    except:
        pass

time: 2.97 ms (started: 2021-11-23 12:31:13 -08:00)


In [19]:
# checking dict
stock_data

{'Company': [],
 'Symbol': [],
 'currentPrice': [],
 'dayHigh': [],
 'dayLow': [],
 '52wkHigh': [],
 '52wkLow': [],
 'dividendRate': []}

time: 1.62 ms (started: 2021-11-23 12:31:13 -08:00)


### Data Frame of S&P 500 Stocks in the News

In [20]:
in_the_news = pd.DataFrame(stock_data)
in_the_news.head()

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate


time: 5.03 ms (started: 2021-11-23 12:31:13 -08:00)


In [21]:
# checking Dtypes
in_the_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       0 non-null      float64
 1   Symbol        0 non-null      float64
 2   currentPrice  0 non-null      float64
 3   dayHigh       0 non-null      float64
 4   dayLow        0 non-null      float64
 5   52wkHigh      0 non-null      float64
 6   52wkLow       0 non-null      float64
 7   dividendRate  0 non-null      float64
dtypes: float64(8)
memory usage: 124.0 bytes
time: 4.44 ms (started: 2021-11-23 12:31:13 -08:00)


## Individual Stock Price Helper Function
Checking the helper function.

In [22]:
h.get_prices('MMM')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,3M Company,MMM,179.37,181.32,178.65,208.95,163.38,5.92


time: 4.45 s (started: 2021-11-23 12:31:13 -08:00)


In [23]:
h.get_prices('AAPL')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,Apple Inc.,AAPL,160.6999,161.8,159.07,165.7,112.59,0.88


time: 2.47 s (started: 2021-11-23 12:31:18 -08:00)


## Options Markets Tests

In [24]:
# creating a fuction to display available expirations
def get_expirations(symbol : str):
    stock = yf.Ticker(symbol)
    return stock.options

time: 605 µs (started: 2021-11-23 12:31:20 -08:00)


In [25]:
get_expirations('AAPL')

('2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-02-18',
 '2022-03-18',
 '2022-04-14',
 '2022-05-20',
 '2022-06-17',
 '2022-07-15',
 '2022-09-16',
 '2023-01-20',
 '2023-03-17',
 '2023-06-16',
 '2023-09-15',
 '2024-01-19')

time: 166 ms (started: 2021-11-23 12:31:20 -08:00)


In [26]:
get_expirations('MMM')

('2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-04-14',
 '2022-06-17',
 '2022-07-15',
 '2023-01-20',
 '2024-01-19')

time: 129 ms (started: 2021-11-23 12:31:21 -08:00)


In [27]:
def options_mrkt_test(symbol : str, expiration : str, option : str):
    stock = yf.Ticker(symbol)
    current_price = stock.info['currentPrice']
    dividend = stock.info['dividendRate']
    url = ('https://www.treasury.gov/resource-center/'
    'data-chart-center/interest-rates/Pages/TextView.aspx?data=yield')
    rates = pd.read_html(url)
    risk_free_rate = rates[1]['3 mo']
    print(f"Current Price: ${current_price}")
    print(f"Dividend: ${dividend}")
    print(f"3 Month TBill Rate: {risk_free_rate.to_string().split()[1]}%")
    opt = stock.option_chain(expiration)
    if option == 'calls':
        return opt.calls.sort_values(by='strike')
    elif option == 'puts':
        return opt.puts.sort_values(by='strike')

time: 1.17 ms (started: 2021-11-23 12:31:21 -08:00)


In [28]:
options_mrkt_test('MMM', '2021-11-26', 'calls' )

Current Price: $179.37
Dividend: $5.92
3 Month TBill Rate: 0.05%


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MMM211126C00170000,2021-11-23 15:50:07,170.0,10.15,9.50,10.05,-1.81,-15.133783,2.0,1,0.489263,True,REGULAR,USD
1,MMM211126C00175000,2021-11-22 17:54:25,175.0,6.25,4.80,5.05,0.00,0.000000,1.0,1,0.298835,True,REGULAR,USD
2,MMM211126C00177500,2021-11-23 15:44:03,177.5,2.60,2.49,2.85,-2.00,-43.478260,1.0,27,0.236092,True,REGULAR,USD
3,MMM211126C00180000,2021-11-23 19:58:58,180.0,1.22,0.94,1.06,-0.04,-3.174600,189.0,421,0.180184,False,REGULAR,USD
4,MMM211126C00182500,2021-11-23 20:04:26,182.5,0.27,0.24,0.32,-0.13,-32.500000,222.0,381,0.177743,False,REGULAR,USD
5,MMM211126C00185000,2021-11-23 19:49:12,185.0,0.11,0.06,0.10,-0.06,-35.294117,117.0,763,0.192391,False,REGULAR,USD
6,MMM211126C00187500,2021-11-23 18:12:30,187.5,0.07,0.03,0.06,0.01,16.666670,40.0,417,0.232430,False,REGULAR,USD
7,MMM211126C00190000,2021-11-23 18:16:06,190.0,0.03,0.02,0.04,-0.01,-25.000000,7.0,370,0.269539,False,REGULAR,USD
8,MMM211126C00192500,2021-11-22 15:32:59,192.5,0.03,0.00,0.07,0.00,0.000000,3.0,24,0.349616,False,REGULAR,USD
9,MMM211126C00195000,2021-11-23 18:32:28,195.0,0.02,0.01,0.04,-0.02,-50.000000,4.0,53,0.371100,False,REGULAR,USD


time: 3.1 s (started: 2021-11-23 12:31:21 -08:00)


In [29]:
h.options_mrkt('MMM', '2021-11-26', 'calls' )

Current Price: $179.37
Dividend: $5.92
3 Month TBill Rate: 0.05%


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MMM211126C00170000,2021-11-23 15:50:07,170.0,10.15,9.50,10.05,-1.81,-15.133783,2.0,1,0.489263,True,REGULAR,USD
1,MMM211126C00175000,2021-11-22 17:54:25,175.0,6.25,4.80,5.05,0.00,0.000000,1.0,1,0.298835,True,REGULAR,USD
2,MMM211126C00177500,2021-11-23 15:44:03,177.5,2.60,2.49,2.85,-2.00,-43.478260,1.0,27,0.236092,True,REGULAR,USD
3,MMM211126C00180000,2021-11-23 19:58:58,180.0,1.22,0.94,1.06,-0.04,-3.174600,189.0,421,0.180184,False,REGULAR,USD
4,MMM211126C00182500,2021-11-23 20:04:26,182.5,0.27,0.24,0.32,-0.13,-32.500000,222.0,381,0.177743,False,REGULAR,USD
5,MMM211126C00185000,2021-11-23 19:49:12,185.0,0.11,0.06,0.10,-0.06,-35.294117,117.0,763,0.192391,False,REGULAR,USD
6,MMM211126C00187500,2021-11-23 18:12:30,187.5,0.07,0.03,0.06,0.01,16.666670,40.0,417,0.232430,False,REGULAR,USD
7,MMM211126C00190000,2021-11-23 18:16:06,190.0,0.03,0.02,0.04,-0.01,-25.000000,7.0,370,0.269539,False,REGULAR,USD
8,MMM211126C00192500,2021-11-22 15:32:59,192.5,0.03,0.00,0.07,0.00,0.000000,3.0,24,0.349616,False,REGULAR,USD
9,MMM211126C00195000,2021-11-23 18:32:28,195.0,0.02,0.01,0.04,-0.02,-50.000000,4.0,53,0.371100,False,REGULAR,USD


time: 2.87 s (started: 2021-11-23 12:31:24 -08:00)


## Dividend Tests

In [30]:
msft = yf.Ticker('MSFT')

time: 3.62 ms (started: 2021-11-23 12:34:22 -08:00)


In [32]:
exdiv = msft.info['exDividendDate']
exdiv

1637107200

time: 2.03 ms (started: 2021-11-23 12:34:41 -08:00)
